# Analisis de datos - Red de tiendas RetailNow
## Trabajo 3: Analisis con Numpy y Pandas

Analisis del rendimiento de las tiendas de RetailNow a partir de datos de ventas, inventarios y satisfaccion del cliente.

## 1. Importar librerias

In [ ]:
import pandas as pd
import numpy as np

## 2. Carga y limpieza de datos

Cargamos los tres archivos CSV y eliminamos filas con valores nulos.

In [ ]:
# cargar los CSV
df_ventas = pd.read_csv("/workspace/sales.csv")
df_inventarios = pd.read_csv("/workspace/inventories.csv")
df_satisfaccion = pd.read_csv("/workspace/satisfaction.csv")

# eliminar filas con valores nulos
df_ventas = df_ventas.dropna()
df_inventarios = df_inventarios.dropna()
df_satisfaccion = df_satisfaccion.dropna()

print("Ventas:", df_ventas.shape)
print("Inventarios:", df_inventarios.shape)
print("Satisfaccion:", df_satisfaccion.shape)

Verificamos la estructura de cada DataFrame:

In [ ]:
print("--- Ventas ---")
print(df_ventas.dtypes)
print()
df_ventas.head()

In [ ]:
print("--- Inventarios ---")
print(df_inventarios.dtypes)
print()
df_inventarios.head()

In [ ]:
print("--- Satisfaccion ---")
print(df_satisfaccion.dtypes)
print()
df_satisfaccion.head()

## 3. Exploracion de datos (Pandas)

### 3.1. Ventas totales por producto y por tienda

In [ ]:
# ventas totales por producto
ventas_por_producto = df_ventas.groupby("Producto")["Cantidad_Vendida"].sum()
print("Ventas totales por producto:")
print(ventas_por_producto)

In [ ]:
# ventas totales por tienda
ventas_por_tienda = df_ventas.groupby("ID_Tienda")["Cantidad_Vendida"].sum()
print("Ventas totales por tienda:")
print(ventas_por_tienda)

### 3.2. Ingresos totales por tienda

In [ ]:
# calcular ingresos = cantidad * precio unitario
df_ventas["Ingresos"] = df_ventas["Cantidad_Vendida"] * df_ventas["Precio_Unitario"]

ingresos_por_tienda = df_ventas.groupby("ID_Tienda")["Ingresos"].sum()
print("Ingresos totales por tienda:")
print(ingresos_por_tienda)

### 3.3. Resumen estadistico de ventas

In [ ]:
# resumen con describe()
print("Resumen estadistico de ventas:")
df_ventas[["Cantidad_Vendida", "Precio_Unitario", "Ingresos"]].describe()

### 3.4. Promedio de ventas por tienda y producto

In [ ]:
# promedio de cantidad vendida por tienda y producto
promedio_tienda_producto = df_ventas.groupby(["ID_Tienda", "Producto"])["Cantidad_Vendida"].mean()
print("Promedio de ventas por tienda y producto:")
print(promedio_tienda_producto)

## 4. Analisis de inventarios (Pandas)

### 4.1. Rotacion de inventarios

La rotacion se calcula dividiendo las ventas totales de cada producto en cada tienda entre el stock disponible.

In [ ]:
# unir ventas con inventarios por tienda y producto
df_rotacion = pd.merge(
    df_ventas.groupby(["ID_Tienda", "Producto"])["Cantidad_Vendida"].sum().reset_index(),
    df_inventarios[["ID_Tienda", "Producto", "Stock_Disponible"]],
    on=["ID_Tienda", "Producto"],
    how="inner"
)

# calcular rotacion
df_rotacion["Rotacion"] = df_rotacion["Cantidad_Vendida"] / df_rotacion["Stock_Disponible"]

print("Rotacion de inventarios por tienda y producto:")
print(df_rotacion)

### 4.2. Tiendas con inventario critico

Filtramos las tiendas donde el porcentaje de productos vendidos es menor al 10% del stock disponible (rotacion < 0.10).

In [ ]:
# filtrar tiendas con rotacion < 10%
inventario_critico = df_rotacion[df_rotacion["Rotacion"] < 0.10]

if len(inventario_critico) == 0:
    print("No hay tiendas con niveles criticos de inventario.")
else:
    print("Tiendas con inventario critico (rotacion < 10%):")
    print(inventario_critico)

## 5. Satisfaccion del cliente (Pandas)

### 5.1. Analisis de satisfaccion y relacion con ventas

In [ ]:
# unir satisfaccion con ingresos por tienda
df_satisfaccion_ventas = pd.merge(
    df_satisfaccion,
    ingresos_por_tienda.reset_index().rename(columns={"Ingresos": "Ingresos_Totales"}),
    on="ID_Tienda",
    how="inner"
)

print("Satisfaccion y ventas por tienda:")
print(df_satisfaccion_ventas)

### 5.2. Tiendas con baja satisfaccion (< 60%)

In [ ]:
# filtrar tiendas con satisfaccion < 60%
baja_satisfaccion = df_satisfaccion_ventas[df_satisfaccion_ventas["Satisfacción_Promedio"] < 60]

if len(baja_satisfaccion) == 0:
    print("No hay tiendas con satisfaccion inferior al 60%.")
else:
    print("Tiendas con baja satisfaccion (< 60%):")
    print(baja_satisfaccion)
    print()
    print("Recomendaciones:")
    for i, fila in baja_satisfaccion.iterrows():
        print(f"  Tienda {fila['ID_Tienda']}: satisfaccion del {fila['Satisfacción_Promedio']}%.")
        print(f"    -> Revisar atencion al cliente y calidad de servicio.")
        print(f"    -> Considerar encuestas detalladas para identificar areas de mejora.")

## 6. Calculos estadisticos con Numpy

### 6.1. Mediana y desviacion estandar de ventas totales

Convertimos la columna de ventas totales a un array de Numpy y calculamos la mediana y la desviacion estandar.

In [ ]:
# calcular ventas totales por tienda (cantidad vendida)
total_ventas = df_ventas.groupby("ID_Tienda")["Cantidad_Vendida"].sum()

# convertir a array de numpy
ventas_array = total_ventas.to_numpy()
print("Array de ventas totales por tienda:", ventas_array)

# mediana
mediana = np.median(ventas_array)
print(f"Mediana de ventas totales: {mediana}")

# desviacion estandar
desviacion = np.std(ventas_array)
print(f"Desviacion estandar de ventas totales: {desviacion:.2f}")

## 7. Simulacion de proyecciones de ventas (Numpy)

Generamos proyecciones de ventas futuras para los proximos 12 meses usando arrays aleatorios de Numpy. Usamos como base la media y desviacion estandar de las ventas actuales.

In [ ]:
# semilla para resultados reproducibles
np.random.seed(42)

# parametros basados en los datos reales
media_ventas = np.mean(ventas_array)
std_ventas = np.std(ventas_array)

# simular ventas para 5 tiendas durante 12 meses
proyecciones = np.random.normal(loc=media_ventas, scale=std_ventas, size=(5, 12))

# redondear a enteros (no tiene sentido vender 35.7 unidades)
proyecciones = np.round(proyecciones).astype(int)

print("Proyecciones de ventas (5 tiendas x 12 meses):")
print(proyecciones)

### 7.1. Estadisticas de las proyecciones

In [ ]:
# estadisticas por tienda
for i in range(len(proyecciones)):
    tienda = proyecciones[i]
    print(f"Tienda {i+1}: media={np.mean(tienda):.1f}, mediana={np.median(tienda):.1f}, std={np.std(tienda):.1f}, min={np.min(tienda)}, max={np.max(tienda)}")

print()
# estadisticas globales
print(f"Media global proyectada: {np.mean(proyecciones):.1f}")
print(f"Desviacion estandar global: {np.std(proyecciones):.1f}")

## 8. Conclusiones

- Se identificaron las tiendas con mayor y menor volumen de ventas e ingresos.
- La rotacion de inventarios permite detectar productos con baja salida que podrian requerir ajustes en la estrategia comercial.
- Las tiendas con satisfaccion inferior al 60% necesitan atencion prioritaria en calidad de servicio.
- Las proyecciones con Numpy permiten estimar escenarios futuros basados en el comportamiento historico de las ventas.